# Co-creating music with SymphonyNet

## Install the requirements

In [ ]:
!git clone https://github.com/symphonynet/SymphonyNet.git

Cloning into 'SymphonyNet'...
remote: Enumerating objects: 93, done.
remote: Counting objects: 100% (15/15), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 93 (delta 3), reused 9 (delta 0), pack-reused 78
Unpacking objects: 100% (93/93), done.


In [ ]:
!cat SymphonyNet/requirements.txt | xargs -n 1 -L 1 pip install

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 776.8 MB 4.8 kB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.13.0+cu116
    Uninstalling torch-1.13.0+cu116:
      Successfully uninstalled torch-1.13.0+cu116
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.14.0+cu116 requires torch==1.13.0, but you have torch 1.7.1 which is incompatible.
torchtext 0.14.0 requires torch==1.13.0, but you have torch 1.7.1 which is incompatible.
torchaudio 0.13.0+cu116 requires torch==1.13.0, but you have torch 1.7.1 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.7 MB 4.7 MB/s 
     |████████████████████████████████| 154 kB 45.1 MB/s 
  

In [ ]:
!sudo apt-get install nvidia-cuda-toolkit nvidia-cuda-dev libcupti-dev

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
The following additional packages will be installed:
  cpp-6 fonts-dejavu-core fonts-dejavu-extra g++-6 gcc-6 gcc-6-base
  libaccinj64-9.1 libasan3 libatk-wrapper-java libatk-wrapper-java-jni
  libcublas9.1 libcudart9.1 libcufft9.1 libcufftw9.1 libcuinj64-9.1
  libcupti-doc libcupti9.1 libcurand9.1 libcusolver9.1 libcusparse9.1
  libgail-common libgail18 libgcc-6-dev libgtk2.0-0 libgtk2.0-bin
  libgtk2.0-common libnppc9.1 libnppial9.1 libnppicc9.1 libnppicom9.1
  libnppidei9.1 libnppif9.1 libnppig9.1 libnppim9.1 libnppist9.1 libnppisu9.1
  libnppitc9.1 libnpps9.1 libnvblas9.1 libnvgraph9.1 libnvrtc9.1
  libnvtoolsext1 libnvvm3 libstdc++-6-dev libthrust-dev libvdpau-dev
  libxxf86dga1 nvidia-cuda-doc nvidia-cuda-gdb nvidia-profiler
  nvidia-visual-profile

# Training pipeline

1. Загружаем MIDI в папку data/midis

In [ ]:
%cd SymphonyNet

/content/SymphonyNet


In [ ]:
!python3 src/preprocess/preprocess_midi.py

starts processing 4 midis with 32 processes
MIDI data preprocessing takes: 5.747195482254028s, 4 samples collected, 0 broken.
Create txt file takes:  0.016820192337036133


In [ ]:
!pwd

/content/SymphonyNet


In [ ]:
!g++ -std=c++11 -pthread -O3 src/musicBPE/fastBPE/main.cc -IfastBPE -o music_bpe_exec

In file included from src/musicBPE/fastBPE/main.cc:1:0:
src/musicBPE/fastBPE/fastBPE.hpp: In function ‘void fastBPE::readText(const char*, std::unordered_map<std::__cxx11::basic_string<char>, unsigned int>&)’:
src/musicBPE/fastBPE/fastBPE.hpp:50:11: warning: ignoring return value of ‘int fscanf(FILE*, const char*, ...)’, declared with attribute warn_unused_result [-Wunused-result]
     fscanf(fin, "%d", &tot);
     ~~~~~~^~~~~~~~~~~~~~~~~
src/musicBPE/fastBPE/fastBPE.hpp:53:15: warning: ignoring return value of ‘int fscanf(FILE*, const char*, ...)’, declared with attribute warn_unused_result [-Wunused-result]
         fscanf(fin, "%s", cur_word);
         ~~~~~~^~~~~~~~~~~~~~~~~~~~~
src/musicBPE/fastBPE/fastBPE.hpp:55:15: warning: ignoring return value of ‘int fscanf(FILE*, const char*, ...)’, declared with attribute warn_unused_result [-Wunused-result]
         fscanf(fin, "%d", &cnt);
         ~~~~~~^~~~~~~~~~~~~~~~~


In [ ]:
!python3 src/preprocess/get_bpe_data.py

reading original txt file...: 4it [00:00, 5785.25it/s]
100% 4/4 [00:00<00:00, 79.91it/s]
learnBPE finished, time elapsed:　0.1564497947692871
100% 597/597 [00:00<00:00, 46509.89it/s]
average mulpi length original:　2.312249443207127, average mulpi length after bpe: 1.0
applyBPE for word finished, time elapsed:　0.020878314971923828
writing bpe data: 100% 4/4 [00:00<00:00, 109.89it/s]
applyBPE for corpus finished, time elapsed:　0.03667807579040527
before tokens: 26617, after tokens: 23671, delta: 0.1106811436300109


**В config.sh добавить BPE = 1**

In [ ]:
!python3 src/fairseq/make_data.py

config.sh: 
BPE=1
IGNORE_META_LOSS=1
PI_LEVEL=2
RECOVER=0
SEED=1998
MAX_POS_LEN=4096
MAXPIECES=10000000
RATIO=4
SOR=4
TOTAL_UPDATES=210000
WARMUP_UPDATES=5000
PEAK_LR=0.0003
BATCH_SIZE=128
MAX_SENTENCES=2
reading...: 4it [00:00, 13888.42it/s]
setting up vocabs: 100% 4/4 [00:00<00:00, 149.55it/s]
max voc idx:  799
sub vocab size: 800 36 16 21 
total pieces: 4, create dict time: 0.19 s
train
measure collection finished, total 163 measures, time elapsed: 0.16820025444030762 s
max cnt in a sample (rel_pos, measure): 29, 80
writing bin file: 100% 2/2 [00:00<00:00, 647.17it/s]
write binary finished, write time elapsed 0.00 s
valid
measure collection finished, total 93 measures, time elapsed: 0.15242314338684082 s
max cnt in a sample (rel_pos, measure): 27, 54
writing bin file: 100% 1/1 [00:00<00:00, 638.99it/s]
write binary finished, write time elapsed 0.00 s
test
measure collection finished, total 187 measures, time elapsed: 0.1593492031097412 s
max cnt in a sample (rel_pos, measure): 35, 1

In [ ]:
!sh train_linear_chord.sh

2022-12-25 09:46:47 | INFO | fairseq_cli.train | Namespace(adam_betas='(0.9, 0.98)', adam_eps=1e-06, add_bos_token=False, all_gather_list_size=16384, arch='linear_transformer_multi', batch_size=2, batch_size_valid=2, best_checkpoint_metric='loss', bf16=False, bpe=None, broadcast_buffers=False, bucket_cap_mb=25, checkpoint_shard_count=1, checkpoint_suffix='_linear_4096_chord_bpe_hardloss1_PI2', clip_norm=0.0, cpu=False, criterion='multiple_loss', curriculum=0, data='data/model_spec/linear_4096_chord_bpe_hardloss1/bin', data_buffer_size=10, dataset_impl=None, ddp_backend='c10d', device_id=0, disable_validation=False, distributed_backend='nccl', distributed_init_method=None, distributed_no_spawn=False, distributed_num_procs=1, distributed_port=-1, distributed_rank=0, distributed_world_size=1, distributed_wrapper='DDP', dropout=0.1, dur_voc_size=36, embed_dim=512, empty_cache_freq=0, end_learning_rate=0.0, evt_voc_size=800, fast_stat_sync=False, find_unused_parameters=False, finetune_from_

In [ ]:
from google.colab import files
files.download('ckpt/checkpoint_last_linear_4096_chord_bpe_hardloss1_PI2.pt')

In [ ]:
a = -1
while True:
  a = a * -1

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Model preparation

In [ ]:
!pwd

/content


In [ ]:
%cd ../

/content


In [ ]:
MAX_POS_LEN = 4096
PI_LEVEL = 2
IGNORE_META_LOSS = 1
RATIO = 4
BPE = "_bpe" # or ""

DATA_BIN=f"linear_{MAX_POS_LEN}_chord{BPE}_hardloss{IGNORE_META_LOSS}"
CHECKPOINT_SUFFIX=f"{DATA_BIN}_PI{PI_LEVEL}"
DATA_BIN_DIR=f"SymphonyNet/data/model_spec/{DATA_BIN}/bin/"
DATA_VOC_DIR=f"SymphonyNet/data/model_spec/{DATA_BIN}/vocabs/"
from SymphonyNet.src.fairseq.gen_utils import process_prime_midi, gen_one, get_trk_ins_map, get_note_seq, note_seq_to_midi_file, music_dict
music_dict.load_vocabs_bpe(DATA_VOC_DIR, 'SymphonyNet/data/bpe_res/' if BPE == '_bpe' else None)

Initialize the model and load pretrained parameters. (You should first save the provided [ckpt file](https://drive.google.com/file/d/1xpkj_qN4MdLRkBdCXmfGjuWWjnTN1Og0/view) into your google drive.)

In [ ]:
!pwd

/content


In [ ]:
from fairseq.models import FairseqLanguageModel
from google.colab import drive
drive.mount('/content/drive')
custom_lm = FairseqLanguageModel.from_pretrained('.', 
    checkpoint_file=f'SymphonyNet/ckpt/checkpoint_last_linear_4096_chord_bpe_hardloss1_PI2.pt', 
    data_name_or_path=DATA_BIN_DIR, 
    user_dir="SymphonyNet/src/fairseq/linear_transformer_inference")

ModuleNotFoundError: ignored

In [ ]:
!pwd

/content


In [ ]:
orig_lm = FairseqLanguageModel.from_pretrained('.', 
    checkpoint_file=f'drive/MyDrive/orig_model.pt', 
    data_name_or_path=DATA_BIN_DIR, 
    user_dir="SymphonyNet/src/fairseq/linear_transformer_inference")

NameError: ignored

In [ ]:
for embedding_layers in ['wEvte', 'wTrke','wDure','wRpe','wMpe']:

  a = getattr(orig_lm.models[0].decoder, embedding_layers)
  setattr(custom_lm.models[0].decoder, embedding_layers, a) 


In [ ]:
m = custom_lm.models[0]
m.cuda()
m.eval()

## Prepare prime MIDI

In [ ]:
!pwd

/content


In [ ]:
%cd SymphonyNet

/content/SymphonyNet


In [ ]:
midi_name = 'test.mid'
max_measure_cnt = 5
max_chord_measure_cnt = 0
prime, ins_label = process_prime_midi(midi_name, max_measure_cnt, max_chord_measure_cnt)

KeyError: ignored

## Generation

In [ ]:
a = 1
while True:
  a = a * -1

KeyboardInterrupt: ignored

In [ ]:
import time
while(True):
  try:
    generated, ins_logits = gen_one(m, prime, MIN_LEN = 1024)
    break
  except Exception as e:
    print(e)
    continue
trk_ins_map = get_trk_ins_map(generated, ins_logits)
note_seq = get_note_seq(generated, trk_ins_map)
timestamp = time.strftime("%m-%d_%H-%M-%S", time.localtime()) 
output_name = f'output_prime{max_measure_cnt}_chord{max_chord_measure_cnt}_{timestamp}.mid'
note_seq_to_midi_file(note_seq, output_name)

  0%|          | 2/3720 [00:00<01:06, 55.84it/s]


Invalid generation: there must be a <bom> before a chord 16
Invalid generation: there must be a <bom> before a chord 16


  0%|          | 7/3720 [00:00<00:49, 74.90it/s]


Invalid generation: there must be a <bom> before a chord 16


  0%|          | 3/3720 [00:00<01:00, 61.48it/s]


...

  0%|          | 1/3720 [00:00<01:38, 37.93it/s]


Invalid generation: there must be a <bom> before a chord 16


  0%|          | 3/3720 [00:00<00:59, 61.98it/s]


Invalid generation: there must be a <bom> before a chord 16


KeyboardInterrupt: ignored

In [ ]:
from google.colab import files
files.download(output_name)

## Audio Display

In [ ]:
!sudo apt install -y fluidsynth

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
The following additional packages will be installed:
  fluid-soundfont-gm libfluidsynth1 libqt5x11extras5 qsynth
Suggested packages:
  fluid-soundfont-gs timidity jackd
The following NEW packages will be installed:
  fluid-soundfont-gm fluidsynth libfluidsynth1 libqt5x11extras5 qsynth
0 upgraded, 5 newly installed, 0 to remove and 20 not upgraded.
Need to get 120 MB of archives.
After this operation, 150 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 fluid-soundfont-gm all 3.1-5.1 [119 MB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libfluidsynth1 amd64 1.1.9-1 [137 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/universe amd64 fluidsynth amd64 1.1.9-1 [20.7 kB]
Get:4 http://archi

In [ ]:
!pip install --upgrade pyfluidsynth

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install pretty_midi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.6 MB 4.5 MB/s 
  Created wheel for pretty-midi: filename=pretty_midi-0.2.9-py3-none-any.whl size=5591954 sha256=54b74862148297b541d8e584df1dc13ba6becef1c3bf30df7a5dfd4984c68a27
  Stored in directory: /root/.cache/pip/wheels/2a/5a/e3/30eeb9a99350f3f7e21258fcb132743eef1a4f49b3505e76b6
Successfully built pretty-midi


In [ ]:
import fluidsynth
import pretty_midi
from IPython import display
_SAMPLING_RATE = 16000
def display_audio(pm: pretty_midi.PrettyMIDI, seconds=300):
  waveform = pm.fluidsynth(fs=_SAMPLING_RATE)
  # Take a sample of the generated waveform to mitigate kernel resets
  waveform_short = waveform[:seconds*_SAMPLING_RATE]
  return display.Audio(waveform_short, rate=_SAMPLING_RATE)

In [ ]:
pm = pretty_midi.PrettyMIDI(output_name)
display_audio(pm)

NameError: ignored